<a href="https://colab.research.google.com/github/sugarforever/Langchain-Tutorials/blob/main/Langchain_AutoGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install langchain==0.0.145

Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install google-search-results

  Preparing metadata (setup.py) ... done
  DEPRECATION: google-search-results is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for google-search-results ... done
Note: you may need to restart the kernel to use updated packages.


In [11]:
from langchain.utilities import SerpAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool
import os
search = SerpAPIWrapper(serpapi_api_key=os.environ.get("SERPAPI_API_KEY"))
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]

In [2]:
from langchain.vectorstores import FAISS
from langchain.docstore import InMemoryDocstore
from langchain.embeddings import OpenAIEmbeddings

In [16]:
%pip install chromadb

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.vectorstores.redis import Redis
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import os

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
persist_directory = ".chromadb"
collection_name = "autogpt"

PINECONE_API_KEY = ''
PINECONE_API_ENV = 'us-west4-gcp'

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_API_ENV)
index_name = "autogpt"
pinecone_index = pinecone.Index(index_name)

#vectorstore = Chroma(
#    collection_name=collection_name,
#    persist_directory=persist_directory,
#    embedding_function=embeddings
#)
#svectorstore.persist()

ModuleNotFoundError: No module named 'redis'

In [13]:
import chromadb
from chromadb.config import Settings
client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet", persist_directory=persist_directory))

client.list_collections()

Using embedded DuckDB with persistence: data will be stored in: .chromadb
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


[Collection(name=langchain), Collection(name=autogpt)]

In [14]:
from langchain.experimental import AutoGPT
from langchain.chat_models import ChatOpenAI

In [18]:
%pip install pinecone_text

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 16.2 MB/s eta 0:00:0000:0100:01
  DEPRECATION: wget is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for wget ... done
  Attempting uninstall: torch
    Found existing installation: torch 2.0.0
    Uninstalling torch-2.0.0:
      Successfully uninstalled torch-2.0.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.15.1
    Uninstalling torchvision-0.15.1:
      Successfully uninstalled torchvision-0.15.1
Note: you may need to restart the kernel to use updated packages.


In [17]:
from langchain.retrievers import PineconeHybridSearchRetriever
from pinecone_text.sparse import BM25Encoder

agent = AutoGPT.from_llm_and_tools(
    ai_name="Nicole",
    ai_role="Assistant",
    tools=tools,
    llm=ChatOpenAI(temperature=0, verbose=True),
    memory=PineconeHybridSearchRetriever(embeddings=embeddings, index=pinecone_index, sparse_encoder=BM25Encoder.default())
)
# Set verbose to be true
agent.chain.verbose = True

In [18]:
agent.run(["write a weather report for Beijing today"])



> Entering new LLMChain chain...


ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'application/json', 'date': 'Thu, 20 Apr 2023 20:37:27 GMT', 'x-envoy-upstream-service-time': '0', 'content-length': '81', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Sparse vector must contain at least one value","details":[]}
